In [1]:
import time
import pymongo 

In [2]:
#!pip install --user pandas==1.0.3

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from pymongo import MongoClient
from splinter import Browser
pd.__version__

'1.0.3'

In [4]:
import feedparser
import pandas as pd
from pandas.io.json import json_normalize
import requests
from config import host, port, username, password, database

In [5]:
import pymysql
import mysql.connector
from sqlalchemy import create_engine

In [6]:
connection = pymysql.connect(host,username,password,database)
cursor = connection.cursor()

In [7]:
month = 'august'
day = '11'
year = '2020'
country = 'venezuela'
url=f'https://news.google.com/rss/search?q=internet+censorship+{country}+{month}+{day}+{year}'
print(url)
news_feed = feedparser.parse(url)

news_scrape=pd.json_normalize(news_feed.entries)

https://news.google.com/rss/search?q=internet+censorship+venezuela+august+11+2020


In [8]:
#Read articles links
title =news_scrape['title']
title = title.to_list()

link = news_scrape['link']
link = link.to_list()

published =  news_scrape['published']
published = published.to_list()

google_search = {'Title':title,'Link':link,'Date':published}

In [9]:
google_scrape_df=pd.DataFrame.from_dict(google_search,orient='index').transpose()

In [10]:
google_scrape_df

,Title,Link,Date
0,VOA Journalists Fly Home After USAGM Fails to ...,https://www.voanews.com/usa/voa-journalists-fl...,"Wed, 26 Aug 2020 01:07:54 GMT"
1,Ertuğrul: how an epic TV series became the 'Mu...,https://www.theguardian.com/tv-and-radio/2020/...,"Wed, 12 Aug 2020 07:00:00 GMT"
2,AP Exclusive: AT&T Under Pressure to Defy Madu...,https://www.usnews.com/news/world/articles/202...,"Fri, 17 Jan 2020 08:00:00 GMT"
3,Trump's Executive Order: What to Know About Se...,https://www.cfr.org/in-brief/trumps-executive-...,"Thu, 04 Jun 2020 07:00:00 GMT"
4,U.S. denies involvement in alleged plot to ove...,https://www.cbsnews.com/video/us-denies-involv...,"Thu, 07 May 2020 07:00:00 GMT"
...,...,...,...
95,UK: Heathrow Airport demands workers to take p...,https://www.wsws.org/en/articles/2020/04/11/he...,"Sat, 11 Apr 2020 07:00:00 GMT"
96,Brazilian disinformation bill threatens freedo...,https://ifex.org/brazilian-disinformation-bill...,"Wed, 24 Jun 2020 07:00:00 GMT"
97,SEP demands military stops harassing its elect...,https://www.wsws.org/en/articles/2020/06/30/se...,"Tue, 30 Jun 2020 07:00:00 GMT"
98,Zoom caught in China censorship crossfire as m...,https://wgnradio.com/news/technology-news/zoom...,"Thu, 11 Jun 2020 07:00:00 GMT"


In [20]:
# Insert DataFrame recrds one by one.
sql = "DROP TABLE IF EXISTS news_scrape"
cursor.execute(sql)

0

In [21]:
sql = "CREATE TABLE news_scrape(id INT PRIMARY KEY AUTO_INCREMENT, title VARCHAR(300) NOT NULL, link VARCHAR(400) NOT NULL, date DATE NOT NULL)"
cursor.execute(sql)

0

In [22]:
cols = ",".join([str(i) for i in google_scrape_df.columns.tolist()])
cols

'Title,Link,Date'

In [24]:
for i,row in google_scrape_df.iterrows():
    sql = str("INSERT INTO news_scrape (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)")
    cursor.execute(sql, tuple(row))
    connection.commit()